In [1]:
# відключимо попередження Anaconda
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

In [2]:
winequality_white_url = 'https://raw.githubusercontent.com/radiukpavlo/intelligent-data-analysis/812a3f66a744bf99069b49e5162084526f740b2d/02_assignments/ida_lab-06_regression-wine/winequality-white.csv'
data = pd.read_csv(winequality_white_url, sep=';')

In [3]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [5]:
y = data['quality']
X = data.drop('quality', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
linreg = LinearRegression(normalize=True)
linreg.fit(X_train_scaled,y_train)

LinearRegression(normalize=True)

In [7]:
linreg_coef = pd.DataFrame(linreg.coef_,data.columns[:-1], columns = ['coeff'])
linreg_coef.sort_values(by = 'coeff', ascending=False)

,coeff
residual sugar,0.538164
pH,0.150036
alcohol,0.129533
fixed acidity,0.097822
sulphates,0.062053
free sulfur dioxide,0.042180
total sulfur dioxide,0.014304
chlorides,0.008127
citric acid,-0.000183
volatile acidity,-0.192260


In [8]:
print("Середньоквадратична помилка (train): %.3f" % mean_squared_error(y_train, linreg.predict(X_train_scaled)))
print("Середньоквадратична помилка (test): %.3f" % mean_squared_error(y_test, linreg.predict(X_test_scaled)))#1

Середньоквадратична помилка (train): 0.558
Середньоквадратична помилка (test): 0.584


In [9]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, random_state=17)

In [10]:
lasso1_coef = pd.DataFrame(lasso1.coef_,data.columns[:-1], columns = ['coeff'])
lasso1_coef.sort_values(by = 'coeff', ascending = False)#2

,coeff
alcohol,0.322425
residual sugar,0.256363
pH,0.067277
free sulfur dioxide,0.043088
sulphates,0.029722
fixed acidity,-0.000000
citric acid,-0.000000
total sulfur dioxide,-0.000000
chlorides,-0.002747
volatile acidity,-0.188479


In [11]:
rf = RandomForestRegressor(random_state=17)
rf.fit(X_train, y_train)

RandomForestRegressor(random_state=17)

In [12]:
print("Середньоквадратична помилка (train): %.3f" % mean_squared_error(y_train, rf.predict(X_train)))
print("Середньоквадратична помилка (test): %.3f" % mean_squared_error(y_test, rf.predict(X_test)))#3

Середньоквадратична помилка (train): 0.053
Середньоквадратична помилка (test): 0.371


In [13]:
rf_mse_cv = np.mean(np.abs(cross_val_score(rf, X_train_scaled, y_train, scoring='neg_mean_squared_error')))

In [14]:
rf_params = {'max_features': [3, 4, 5],
             'min_samples_leaf': [1, 3, 5, 7],
             'max_depth': [5, 7, 10]}

locally_best_rf = GridSearchCV(rf, rf_params,n_jobs=-1)
locally_best_rf.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestRegressor(random_state=17), n_jobs=-1,
             param_grid={'max_depth': [5, 7, 10], 'max_features': [3, 4, 5],
                         'min_samples_leaf': [1, 3, 5, 7]})

In [15]:
locally_best_rf.best_params_, locally_best_rf.best_score_

({'max_depth': 10, 'max_features': 4, 'min_samples_leaf': 1},
 0.4344246838722469)

In [16]:
rf_importance = pd.DataFrame(rf.feature_importances_, data.columns[:-1], columns = ['coeff'])
rf_importance.sort_values(by = 'coeff', ascending = True)#4

,coeff
density,0.056540
citric acid,0.058904
sulphates,0.059056
fixed acidity,0.061918
chlorides,0.063729
total sulfur dioxide,0.067204
residual sugar,0.070903
pH,0.071950
free sulfur dioxide,0.116562
volatile acidity,0.125960


In [17]:
#5 ПЕРЕФРАЗИРУЙ ТЕКСТ НИЖЧЕ
Після побудови моделей, можна зробити такі висновки, як:
Середньоквадратичні помилки моделі лінійної регресії та LASSO-регресії майже однакові,
якщо подивитися на навчальну вибірку (0.558 і 0.564) та тестову вибірку (0.584 і 0.574).
З цього можна робити висновки, щодо якості побудованих моделей.
В свою чергу випадковий ліс має значно менші середньоквадратичні помилки відносно 
своїх конкурентів на обох вибірках: навчальна вибірка (0.053) та тестова вибірка (0.371).
Щодо ознак, що впливали на якість вина:
У лінійної регресії найбільше впливало - residual sugar, а наймешне - density.
У LASSO-регресії найбільше впливало - alcohol, а наймешне - density.
У випадкового лісу найбільше впливало - alcohol, а наймешне - density.

SyntaxError: invalid syntax (1355199993.py, line 2)